<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/ArabicPIIRedaction/blob/main/fine-tuning_SLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
import pandas as pd

## Testing data
- Original Competition Data

In [ ]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

,source,target,dialect
0,الأستاذ أدولفوس ريغان زييمان، بصفتك المدير الت...,الأستاذ [MASK]، بصفتك المدير التنفيذي الرئيسي ...,EGYPT
1,مرحبًا يا هانا، ممكن لو سمحتي تتحققي من التداع...,مرحبًا يا [MASK]، ممكن لو سمحتي تتحققي من التد...,EGYPT
2,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,SAUDI
3,عزيزي ديفان، مطلوب تقديم عرض تقديمي على مستوى ...,عزيزي [MASK]، مطلوب تقديم عرض تقديمي على مستوى...,EGYPT
4,ممكن كمان نعمل جلسة عن كيفية إدارة التوتر والت...,The sentence doesn't contain any personal or s...,LEVANTINE


## Training data

In [ ]:
from google.colab import drive
import random

### Loading Data

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# All Data

data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ALL_DATA.csv"

train = pd.read_csv(data_path)

In [ ]:
print(len(train))
train.head()

328597


,source,dialect,tokens,tags,clean_source
0,انضم لينا في المعرض العلمي عند الإحداثيات <!NE...,eg,"['انضم', 'لينا', 'في', 'المعرض', 'العلمي', 'عن...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEARBYG...",انضم لينا في المعرض العلمي عند الإحداثيات [-18...
1,انضم لمجتمعنا أونلاين للصحة النفسية والرفاهية....,eg,"['انضم', 'لمجتمعنا', 'أونلاين', 'للصحة', 'النف...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",انضم لمجتمعنا أونلاين للصحة النفسية والرفاهية....
2,<!FIRSTNAME_1!> لورينزا </!FIRSTNAME_1!>، وجود...,eg,"['لورينزا', '،', 'وجودك', 'في', 'الندوة', '""',...","['B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', ...","لورينزا، وجودك في الندوة ""اختيارات الغذاء الصح..."
3,شركة ناشئة في مجال الحلول بتدور على توظيف <!JO...,eg,"['شركة', 'ناشئة', 'في', 'مجال', 'الحلول', 'بتد...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-JO...",شركة ناشئة في مجال الحلول بتدور على توظيف منسق...
4,عزيزي <!FIRSTNAME_1!> جيف </!FIRSTNAME_1!>، اح...,eg,"['عزيزي', 'جيف', '،', 'احنا', 'لقينا', 'نشاط',...","['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', ...",عزيزي جيف، احنا لقينا نشاط مشبوه ممكن يكون حصل...


In [ ]:
print(train['dialect'].unique())
print(train['dialect'].value_counts())

['eg' 'en' 'levantine' 'msa' 'saudi' 'LEVANTINE' '0']
dialect
msa          199364
levantine     33296
eg            33021
saudi         32948
en            20000
LEVANTINE      9967
0                 1
Name: count, dtype: int64


In [ ]:
train.loc[train['dialect']=='0', 'dialect'] = 'eg'
print(train['dialect'].value_counts())

dialect
msa          199364
levantine     33296
eg            33022
saudi         32948
en            20000
LEVANTINE      9967
Name: count, dtype: int64




---



In [ ]:
# Data augmented with Arabic names and locations

names_loc_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/names_locations_augmented_data.csv")

print(len(names_loc_data))
names_loc_data.head()

47677


,source,dialect,tokens,tags,clean_source
0,رح نحط قضاة عند الجناح الجنوبي الشرقي من مكان ...,levantine,"['رح', 'نحط', 'قضاة', 'عند', 'الجناح', 'الجنوب...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",رح نحط قضاة عند الجناح الجنوبي الشرقي من مكان ...
1,مساء الخير، خلال اجتماعنا السنوي في <!CITY_1!>...,msa,"['مساء', 'الخير', '،', 'خلال', 'اجتماعنا', 'ال...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CITY', ...",مساء الخير، خلال اجتماعنا السنوي في منطقة الاو...
2,بالنسبة للمخزون في فرع <!ZIPCODE_1!> 52884 </!...,LEVANTINE,"['بالنسبة', 'للمخزون', 'في', 'فرع', '52884', '...","['O', 'O', 'O', 'O', 'B-ZIPCODE', 'O', 'O', 'O...",بالنسبة للمخزون في فرع 52884، معدات العناية ال...
3,يرجى التأكد من أن سجل المريض الأول كامل وجاهز ...,saudi,"['يرجى', 'التأكد', 'من', 'أن', 'سجل', 'المريض'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",يرجى التأكد من أن سجل المريض الأول كامل وجاهز ...
4,هذا إشعار لـ <!FIRSTNAME_1!> آية </!FIRSTNAME_...,msa,"['هذا', 'إشعار', 'لـ', 'آية', 'عضل', '.', 'يمك...","['O', 'O', 'O', 'B-FIRSTNAME', 'B-LASTNAME', '...",هذا إشعار لـ آية عضل. يمكن تحويل المبلغ المستح...


In [ ]:
train = pd.concat([train, names_loc_data], ignore_index=True)
print(len(train))
train['dialect'].value_counts()

376274


,count
dialect,
msa,208928
levantine,42830
eg,42592
saudi,42458
en,20000
LEVANTINE,19466


In [ ]:
# Shuffle the data

train = train.sample(frac=1).reset_index(drop=True)

### Cleaning and preprocessing

In [ ]:
idx = random.randint(0, len(train))
idx

321874

In [ ]:
print(train.tokens[idx])
print(type(train.tokens[idx]))
print(train.tags[idx])
print(type(train.tags[idx]))
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['هلا', 'Trans', '،', 'تم', 'الدفع', 'لصندوق', 'التعليم', 'حقك', 'بنجاح', '.', 'تفاصيل', 'المعاملة', 'هي', 'كالتالي', ':', 'رقم', 'البطاقة', '1108568956295853', '،', 'المبلغ', '₦', '2894163782469196', '.']
<class 'str'>
['O', 'B-GENDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CREDITCARDNUMBER', 'O', 'O', 'B-CURRENCYSYMBOL', 'B-MASKEDNUMBER', 'O']
<class 'str'>
205
167


In [ ]:
import re
import ast

In [ ]:
tokens_list = ast.literal_eval(train.tokens[idx])
print(tokens_list)
print(type(tokens_list), len(tokens_list))

tags_list = ast.literal_eval(train.tags[idx])
print(tags_list)
print(type(tags_list), len(tags_list))

['هلا', 'Trans', '،', 'تم', 'الدفع', 'لصندوق', 'التعليم', 'حقك', 'بنجاح', '.', 'تفاصيل', 'المعاملة', 'هي', 'كالتالي', ':', 'رقم', 'البطاقة', '1108568956295853', '،', 'المبلغ', '₦', '2894163782469196', '.']
<class 'list'> 23
['O', 'B-GENDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CREDITCARDNUMBER', 'O', 'O', 'B-CURRENCYSYMBOL', 'B-MASKEDNUMBER', 'O']
<class 'list'> 23


In [ ]:
# Convert tokens and tags columns to lists

train['tokens'] = train["tokens"].apply(ast.literal_eval)
train['tags'] = train['tags'].apply(ast.literal_eval)

In [ ]:
print(train.tokens[idx])
print(type(train.tokens[idx]))
print(train.tags[idx])
print(type(train.tags[idx]))
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['عزيزي', 'الرجل', 'ذو', 'الجنس', 'الآخر', '،', 'تمت', 'معالجة', 'معاملتك', 'بنجاح', '.', 'تم', 'تحديث', 'الحساب', 'حساب', 'القرض', 'الشخصي', '.', 'إذا', 'عندك', 'أي', 'أسئلة', '،', 'لا', 'تتردد', 'إنك', 'تتواصل', 'معنا', '.']
<class 'list'>
['O', 'B-GENDER', 'I-GENDER', 'I-GENDER', 'I-GENDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ACCOUNTNAME', 'I-ACCOUNTNAME', 'I-ACCOUNTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
<class 'list'>
29
29


In [ ]:
# Checking if there is mismatching lengths between tokens and tags

length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


In [ ]:
# Initialize a list to store discrepancy information
discrepancy_data = []

# Creating a new masked sentence using the correct masker function
train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask, discrepancy_data), axis=1)

# Convert the list of discrepancy data into a DataFrame
discrepancy_df = pd.DataFrame(discrepancy_data)

AttributeError: 'str' object has no attribute 'get'

In [ ]:
tags_to_mask = ['PHONEIMEI',
 'VEHICLEVRM',
 'LITECOINADDRESS',
 'CREDITCARDNUMBER',
 'DATE',
 'NEARBYGPSCOORDINATE',
 'BITCOINADDRESS',
 'GENDER',
 'PERSONNAME',
 'JOBTITLE',
 'TIME',
 'CURRENCY',
 'BIC',
 'MASKEDNUMBER',
 'STREET',
 'MAC',
 'DOB',
 'SECONDARYADDRESS',
 'CREDITCARDISSUER',
 'ZIPCODE',
 'USERAGENT',
 'CURRENCYSYMBOL',
 'JOBTYPE',
 'BUILDINGNUMBER',
 'AGE',
 'MIDDLENAME',
 'CREDITCARDINUMBER',
 'ACCOUNTNUMBER',
 'PIN',
 'FIRSTNAME',
 'ORDINALDIRECTION',
 'PASSWORD',
 'PHONENUMBER',
 'IPV4',
 'CREDITCARDCVV',
 'USERNAME',
 'HEIGHT',
 'CURRENCYCODE',
 'ACCOUNTNAME',
 'IBAN',
 'AMOUNT',
 'PREFIX',
 'VEHICLEVIN',
 'SEX',
 'EMAIL',
 'ETHEREUMADDRESS',
 'IPV6',
 'SSN',
 'URL',
 'LASTNAME',
 'CURRENCYNAME',
 'IP']

In [ ]:
def masker(row, tags_to_mask, discrepancy_list):
    """
    Constructs a masked sentence and records discrepancies, attempting to preserve original formatting.

    Args:
    row: A pandas DataFrame row with 'tokens', 'tags', and 'clean_source' columns.
    tags_to_mask: List of tag entity types to mask.
    discrepancy_list: A list to append discrepancy details.

    Returns:
    The constructed sentence with masked tokens for the given row, attempting to preserve spacing.
    """
    tokens = row['tokens']
    tags = row['tags']
    clean_source = row['clean_source']
    masked_sentence = ""
    current_position = 0

    for i in range(len(tokens)):
        token = tokens[i]
        tag = tags[i]

        # Find the position of the current token in the original string starting from the last processed position
        start_index = clean_source.find(token, current_position)

        if start_index != -1:
            # Append the text from the current position up to the start of the token
            masked_sentence += clean_source[current_position:start_index]

            if tag[2:] in tags_to_mask:
                masked_sentence += '[MASK]'
            else:
                masked_sentence += token

            # Update the current position to the end of the current token
            current_position = start_index + len(token)
        else:
            # If token not found at or after current_position, it indicates a discrepancy
            print(f"Warning: Token '{token}' not found in clean_source at or after position {current_position}\n")

            # Record the discrepancy
            discrepancy_list.append({
                'clean_source': clean_source,
                'tokens': tokens,
                'tags': tags,
                'discrepancy_token': token,
                'discrepancy_position': current_position
            })

            # You might choose how to handle the token that wasn't found -
            # here, we'll just append it unmasked to keep the process going
            masked_sentence += token
            # We don't update current_position based on this token as it wasn't found correctly

    # Append any remaining text after the last token
    masked_sentence += clean_source[current_position:]
    return masked_sentence

In [ ]:
# Initialize a list to store discrepancy information
discrepancy_data = []

# Creating a new masked sentence
train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask, discrepancy_data), axis=1)

# Convert the list of discrepancy data into a DataFrame
discrepancy_df = pd.DataFrame(discrepancy_data)

In [ ]:
print(f"{len(discrepancy_df)} discrepancies are found")

107 discrepancies are found


In [ ]:
print(f"{len(discrepancy_df.clean_source.unique())} sentences are detected as discrepancies.")

7 sentences are detected as discrepancies.


In [ ]:
# Drop discrepancies

print(f"Original number of rows: {len(train)}")

train = train[~train['clean_source'].isin(discrepancy_df['clean_source'])]

print(f"Number of rows after dropping discrepancies: {len(train)}")

Original number of rows: 376274
Number of rows after dropping discrepancies: 376267


In [ ]:
train[['clean_source','masked_source']]

,clean_source,masked_source
0,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...
1,عزيزي جادون، إن بحثك حول دول المراقبة وانتهاكا...,عزيزي [MASK]، إن بحثك حول دول المراقبة وانتهاك...
2,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...
3,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...
4,في 29067 جزر راث، شيد مؤخرا مركز جديد للتربية ...,في [MASK] [MASK] [MASK]، شيد مؤخرا مركز جديد ل...
...,...,...
376269,تم تسجيل أشرف في معسكر STEM. تم تحصيل الرسوم م...,تم تسجيل [MASK] في معسكر STEM. تم تحصيل الرسوم...
376270,بالنسبة للمسابقة العلمية، دور على IPV6 5bf9:9d...,بالنسبة للمسابقة العلمية، دور على IPV6 [MASK][...
376271,انتبه جاكي ريبين، يمكن العثور على معلومات بشأن...,انتبه [MASK] [MASK]، يمكن العثور على معلومات ب...
376272,جمعنا بيانات من جبل طابور. للوصول إلى بيانات إ...,جمعنا بيانات من جبل طابور. للوصول إلى بيانات إ...


In [ ]:
train_final = train[['clean_source', 'masked_source']].copy()
train_final.rename(columns={'clean_source': 'source', 'masked_source': 'target'}, inplace = True)

train_final.head()

# Save the train data after the updates
train_final.to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

# Fine-tuning

### Loading Data

In [1]:
import pandas as pd
from google.colab import drive
import random
import re
from IPython.display import display, HTML

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv")

In [4]:
def merge_consecutive_masks(text):
    """Merges consecutive occurrences of '[MASK]' in a string, preserving a trailing whitespace if present."""
    return re.sub(r'(\[MASK\](\s*)){2,}', r'[MASK]\2', text)

def display_ar_eng(text):
    """Display Arabic and English text in a readable format"""
    display(HTML(f'<div dir="rtl" style="font-size:18px; line-height:1.8; font-family: "Arial", sans-serif;">{text}</div>'))

In [5]:
train_data['target'] = train_data['target'].apply(merge_consecutive_masks)

display_ar_eng(train_data['source'][0])
display_ar_eng(train_data['target'][0])

## Gemma 3 (4B)

### Loading model

In [6]:
!pip install -qU unsloth
from unsloth import FastLanguageModel, FastModel
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt",
    max_seq_length = 2048,
    load_in_4bit = False, # LoRA 16-bit
    dtype = torch.float16,
    full_finetuning = False
)

==((====))==  Unsloth 2025.6.2: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [8]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 101,
    use_gradient_checkpointing = "unsloth" # reduce memory usage by an extra 30% and support extremely long context finetunes
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


### Formatting data

In [9]:
from datasets import Dataset

In [10]:
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Mask the PII words in the given source sentence as "[MASK]".

### Input:
{}

### Response:
{}"""

In [11]:
train_data['text'] = train_data.apply(
    lambda row: prompt.format(row['source'], row['target']),
    axis=1
)

display(train_data[['source', 'target', 'text']].head())

,source,target,text
0,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,يرجى تذكر إن رحلتنا التعليمية في الهواء الطلق ...,Below is an instruction that describes a task....
1,عزيزي جادون، إن بحثك حول دول المراقبة وانتهاكا...,عزيزي [MASK]، إن بحثك حول دول المراقبة وانتهاك...,Below is an instruction that describes a task....
2,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,رح يتم تحديث شبكة الـ WiFi الخاصة بمختبر الأبح...,Below is an instruction that describes a task....
3,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,يقدم العلاج المهني لآلام الكتف في ولاية بنسلفا...,Below is an instruction that describes a task....
4,في 29067 جزر راث، شيد مؤخرا مركز جديد للتربية ...,في [MASK]، شيد مؤخرا مركز جديد للتربية الجنسية...,Below is an instruction that describes a task....


In [12]:
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(train_data)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch training
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/376267 [00:00<?, ? examples/s]

## SILMA

### SILMA-2B-4bit

In [ ]:
# !pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0-bnb-4bit" # Quantized version
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config,
)

In [ ]:
messages = [
    {"role": "user", "content": "كم عدد أيام الأسبوع؟"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

input_ids

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=256)

In [ ]:
tokenizer.decode(outputs[0]).split("<start_of_turn>")[-1]

### SILMA-2B

In [ ]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model="silma-ai/SILMA-Kashif-2B-Instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

In [ ]:
messages = [
    {"role": "user",
     "content": """ Mask the PII as [masked] in the given source sentence

     source: I sent my email address M.helmy@nu.edu.eg to my supervisor dr. gahda khoriba in Nile University.

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
response

In [ ]:
messages = [
    {"role": "user",
     "content": f""" Mask the PII as [masked] in the given source sentence

     source: {df["source"][0]}

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
print(df["source"][0])
print(response)

# Next

- Continue fine-tuning

# Gemi

### Training the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # adjust this based on your needs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
    ),
)

trainer.train()